<a href="https://colab.research.google.com/github/Tariquzzaman-faisal/VITD/blob/main/LSTM_no_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GlobalMaxPool1D, Input, Flatten, MaxPooling1D, SpatialDropout1D, Activation

from keras.callbacks import EarlyStopping

from numpy import array
from sklearn.metrics import classification_report

import gensim
from gensim import models
from gensim.models import Word2Vec
import pandas as pd
import numpy as np

In [ ]:
train_dataset = pd.read_csv("/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/dataset/task datasets/original/train.csv")
val_dataset = pd.read_csv("/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/dataset/task datasets/original/dev.csv")
test_dataset = pd.read_csv("/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/dataset/task datasets/original/test.csv")

In [ ]:
train_x = train_dataset['text']
train_y = train_dataset['label']

val_x = val_dataset['text']
val_y = val_dataset['label']

test_x = test_dataset['text']
test_y = test_dataset['label']

In [ ]:
tokenizer=Tokenizer(oov_token = "<OOV>", split=' ', num_words=max_words) # Splitting text based on whitespace and adding "Out of vocabulary"
tokenizer.fit_on_texts(train_x) # Using the tokenizer on out train dataset to tokenize the train dataset
train_encoded=tokenizer.texts_to_sequences(train_x)
# print(train_encoded)
train_padded= pad_sequences(train_encoded, padding='post', maxlen=256)
# print(train_padded)

In [ ]:
# padding df_test
test_encoded=tokenizer.texts_to_sequences(test_x)
test_padded= pad_sequences(test_encoded, padding='post', maxlen=train_padded.shape[1])

In [ ]:
# padding df_validation
val_encoded=tokenizer.texts_to_sequences(val_x)
val_padded= pad_sequences(val_encoded, padding='post', maxlen=train_padded.shape[1])

In [ ]:
earlystop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="min",
    restore_best_weights=True,
)

In [ ]:
from tensorflow.keras.layers import Layer, Embedding, Bidirectional, LSTM, GlobalMaxPool1D, Dense
from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as K

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from tensorflow.keras.models import Sequential

# Define hyperparameters
max_words = 5000
embedding_dim = 128
lstm_units = 150
num_classes = 3

# Define the LSTM model
model = Sequential([
    # Add an Embedding layer
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=train_padded.shape[1]),
    SpatialDropout1D(0.3),  # Dropout layer to prevent overfitting

    # Add an LSTM layer
    LSTM(lstm_units),

    # Add a Dense layer for classification
    Dense(num_classes, activation='softmax'),
], name="Text_Classification_Model")

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "Text_Classification_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 256, 128)          640000    
                                                                 
 spatial_dropout1d_3 (Spati  (None, 256, 128)          0         
 alDropout1D)                                                    
                                                                 
 lstm_3 (LSTM)               (None, 150)               167400    
                                                                 
 dense_3 (Dense)             (None, 3)                 453       
                                                                 
Total params: 807853 (3.08 MB)
Trainable params: 807853 (3.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Train the model with EarlyStopping
batch_size = 32  # Adjust as needed
epochs = 100  # Adjust as needed

history = model.fit(
    train_padded,
    train_y,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(val_padded, val_y),
    callbacks=[earlystop_callback],  # Add the EarlyStopping callback
)

Epoch 1/100


InvalidArgumentError: ignored